In [5]:
import os
import regex as re

In [6]:
# Formatter for transcripts

def file_concat_suffix(file_name, suffix):
    return file_name.replace('.txt', '') + "_" + suffix + ".txt"

In [7]:
def transcript_omitter(file):
    omit_lines = []
    wavs = os.listdir("wavs/")
    
    with open(file) as f:
        text = f.readlines()
    f.close()

    for index in range(len(text)):
        line = text[index]

        if "|" not in line:
            omit_lines.append(index)
            continue

        wav, transcript = line.split("|")

        if wav not in wavs:
            omit_lines.append(index)
            continue
        
        if len(transcript) > 400:
            omit_lines.append(index)
    
    for omit_index in sorted(omit_lines, reverse = True):
        del text[omit_index]

    new_text = ''.join(text)

    new_file_name = file_concat_suffix(file, "omitted")
    new_file = open(new_file_name, 'w')

    new_file.write(new_text)
    new_file.close()

transcript_omitter('Mixed.txt',)

In [9]:
def transcript_formatter(file, omitted = True):
    if omitted:
        with open(file_concat_suffix(file, "omitted")) as f:
            text = f.read()
        f.close()
        
    else:
        with open(file) as f:
            text = f.read()
        f.close()

    replacements = {'.wav' : '',
                    '%' : 'percent',
                    '’' : "'",
                    '–' : '-',
                    # ')' : '',
                    # '(' : '',
                    '‘' : '',
                    # '_' : ' ',
                    '[laughter]' : 'hahaha',
                    '[Laughter]' : 'hahaha',
                    '<laughs>' : 'hahaha',
                    '<laugh>' : 'hahaha',
                    '<' : '',
                    '>' : '',
}

    for word, replacement in replacements.items():
        text = text.replace(word, replacement)
    
    new_file_name = file_concat_suffix(file, "formatted")
    new_file = open(new_file_name, 'w')
    
    new_file.write(text)
    new_file.close()

transcript_formatter('Mixed.txt', True)

In [8]:
from pydub import AudioSegment
from os import listdir
from os.path import isfile, join
import librosa
import soundfile as sf

# Turn all stereo to mono
wavs = [join("wavs/", file) for file in listdir("wavs") if isfile(join("wavs/", file))]

for wav_path in wavs:
    # print(wav_path)

    try:
        sound = AudioSegment.from_wav(wav_path)
        sound = sound.set_frame_rate(22050)
        sound = sound.set_channels(1)
        sound.export(wav_path, format="wav", )
    except:
        y, sr = librosa.load(wav_path, sr=22050)
        sf.write(wav_path, librosa.to_mono(y), 22050, format = 'wav')

In [14]:
from pydub.utils import mediainfo
>>> info = mediainfo("wavs/Tyler (145).wav")
>>> print(info['sample_rate'])

22050
